In [1]:
import numpy as np

import seaborn as sns

In [4]:
data = np.load('Simulations/Epsilon0.1Alpha0.000333333333333Iterations10000000.npy')

data.shape

(10, 10, 2, 2)

In [ ]:
Soft_Stand = data[:, :, 1, 1] 
Hard_Stand = data[:, :, 1, 1] 
Soft_Hit = data[:, :, 1, 1] 
Hard_Hit = data[:, :, 1, 1] 